# Lightgbm Model

In [1]:
import pandas as pd
from xgboost import XGBRegressor, callback
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import numpy as np
from tqdm import tqdm
from datetime import datetime
import pytz
import json
import joblib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from lightgbm import LGBMRegressor


In [2]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v16.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v16.csv")

In [3]:
# Function to drop columns with more than 40% missing values, except for 'tow' in the submission set
def drop_columns_above_threshold(df, threshold=40, preserve_columns=None):
    if preserve_columns is None:
        preserve_columns = []
    
    missing_percentage = df.isna().mean() * 100
    cols_to_keep = missing_percentage[missing_percentage <= threshold].index.tolist()
    
    # Ensure columns in preserve_columns are kept even if they exceed the threshold
    cols_to_keep.extend([col for col in preserve_columns if col in df.columns])
    
    df = df[cols_to_keep]
    return df

# Applying the function to challenge_set_updated
challenge_set_updated = drop_columns_above_threshold(challenge_set_updated)

# Applying the function to submission_set_updated, keeping 'tow'
submission_set_updated = drop_columns_above_threshold(submission_set_updated, preserve_columns=['tow'])

In [4]:
def clean_data_better(df, threshold=1e10):
    # Replace inf and -inf with NaN using vectorized operations
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Mask values above the threshold with NaN directly using vectorized operations
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].mask(df[numeric_columns].abs() > threshold)
    
    # Fill NaNs using a combined approach - first forward fill, then median
    df[numeric_columns] = df[numeric_columns].fillna(method='ffill').fillna(df.median())
    
    return df

# Applying the improved cleaning function
challenge_set_updated = clean_data_better(challenge_set_updated)
submission_set_updated = clean_data_better(submission_set_updated)

/tmp/ipykernel_13810/653332527.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[numeric_columns] = df[numeric_columns].fillna(method='ffill').fillna(df.median())
/tmp/ipykernel_13810/653332527.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[numeric_columns] = df[numeric_columns].fillna(method='ffill').fillna(df.median())


In [5]:
# If necessary change this part to test the model before the training process
df = challenge_set_updated.iloc[:,:]

# Separating features and target variable
X = df.drop('tow', axis=1)
y = df['tow']

n_jobs = os.cpu_count() // 2

In [6]:
# Split the data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets for early stopping
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the best parameters for LightGBM
best_params = {
    'subsample': 1.0,
    'lambda_l2': 0.46415888336127775,  # reg_lambda in LightGBM
    'lambda_l1': 0.166810053720005,    # reg_alpha in LightGBM
    'min_child_weight': 4,             # This corresponds to min_data_in_leaf in LightGBM
    'max_depth': 13,
    'learning_rate': 0.01,
    'colsample_bytree': 0.6            # same as feature_fraction in LightGBM
}

# Initialize the LightGBM model with the best parameters
best_model = LGBMRegressor(
    **best_params,
    objective='regression',
    random_state=42,
    n_estimators=10_000_000,  # Set a high value to allow early stopping to find the best n_estimators
    n_jobs=n_jobs,
    metric="rmse",            # Use rmse as the evaluation metric
    early_stopping_rounds=20  # Early stopping based on validation performance
)

# Train the model on the training data with early stopping using the validation set
best_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)])

# Update best_params with the best number of estimators found during early stopping
best_params['n_estimators'] = best_model.best_iteration_  # No need to add 1, LightGBM handles this

# Evaluate the final model on the test set
y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Best Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")
print(f"Updated best_params: {best_params}")

[LightGBM] [Warning] lambda_l2 is set=0.46415888336127775, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.46415888336127775
[LightGBM] [Warning] early_stopping_round is set=20, early_stopping_rounds=20 will be ignored. Current value: early_stopping_round=20
[LightGBM] [Warning] lambda_l1 is set=0.166810053720005, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.166810053720005
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l2 is set=0.46415888336127775, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.46415888336127775
[LightGBM] [Warning] lambda_l1 is set=0.166810053720005, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.166810053720005
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041493 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36673
[LightGBM] [Info] Number of data points in the train se

In [8]:
# Save R², RMSE, and hyperparameters
results = {
    'R2': float(r2),
    'RMSE': float(rmse),
    'Best Parameters': {key: (int(value) if isinstance(value, np.integer) else float(value)
                              if isinstance(value, np.floating) else value)
                        for key, value in best_params.items()}
}

# Set timezone to São Paulo (UTC-3)
saopaulo_tz = pytz.timezone('America/Sao_Paulo')
timestamp = datetime.now(saopaulo_tz).strftime('%Y%m%d_%H%M%S')

# Define logs directory, and create them if they don't exist
logs_dir = 'logs'
os.makedirs(logs_dir, exist_ok=True)

# Define file paths within the respective directories
results_file = os.path.join(logs_dir, f'model_results_{timestamp}.txt')

# Save the results to a TXT file
with open(results_file, 'w') as file:
    file.write(f"R2: {results['R2']}\n")
    file.write(f"RMSE: {results['RMSE']}\n")
    file.write("Best Parameters:\n")
    for param, value in results['Best Parameters'].items():
        file.write(f"  {param}: {value}\n")

print(f"Results saved to {results_file}")

Results saved to logs/model_results_20241017_142406.txt


In [9]:
# Display evaluation metrics
print(f"Final Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")

Final Model Performance - R^2 Score: 0.9975, RMSE: 2652.7998


In [11]:
# Define models directory, and create them if they don't exist
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

final_model = LGBMRegressor(**best_params, objective='regression', random_state=42, n_jobs=n_jobs)

# Train the model on the entire training+validation+set data
final_model.fit(X, y)

print("Final model trained successfully using all available data.")

[LightGBM] [Warning] lambda_l2 is set=0.46415888336127775, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.46415888336127775
[LightGBM] [Warning] lambda_l1 is set=0.166810053720005, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.166810053720005
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l2 is set=0.46415888336127775, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.46415888336127775
[LightGBM] [Warning] lambda_l1 is set=0.166810053720005, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.166810053720005
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.067328 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36681
[LightGBM] [Info] Number of data points in the train set: 369013, number of used features: 524
[LightGBM] [Info] Start training from score 79482.257206
Final model trained successfully usin

In [12]:
# Define file paths within the respective directories
model_file = os.path.join(models_dir, f'trained_model_{timestamp}.joblib')

# Save the trained model to a file in the models folder
joblib.dump(final_model, model_file)
print(f"Model saved to {model_file}")

Model saved to models/trained_model_20241017_142406.joblib


In [13]:
submission_set_updated.T

,0,1,2,3,4,5,6,7,8,9,...,105949,105950,105951,105952,105953,105954,105955,105956,105957,105958
adep_height_1,1063.0,41.0,117.0,220.0,358.0,1688.0,42.0,1684.0,17.0,18402.0,...,193.0,638.0,53.0,12.0,87.0,938.0,25.0,975.0,311.0,311.0
adep_height_10,1288.0,491.0,667.0,595.0,808.0,1863.0,467.0,1684.0,492.0,18002.0,...,143.0,143.0,228.0,312.0,412.0,1063.0,475.0,1225.0,311.0,311.0
adep_height_2,1063.0,66.0,117.0,220.0,408.0,1688.0,67.0,1684.0,67.0,18402.0,...,193.0,713.0,78.0,37.0,87.0,988.0,25.0,975.0,311.0,311.0
adep_height_3,1063.0,141.0,217.0,295.0,458.0,1688.0,117.0,1684.0,117.0,18402.0,...,193.0,763.0,103.0,87.0,87.0,1013.0,100.0,1050.0,311.0,311.0
adep_height_4,1213.0,191.0,217.0,345.0,508.0,1688.0,167.0,1684.0,192.0,18002.0,...,193.0,763.0,128.0,87.0,212.0,1013.0,150.0,1050.0,311.0,311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ades_geo_cluster_16,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ades_geo_cluster_17,True,False,False,False,True,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
ades_geo_cluster_18,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ades_geo_cluster_19,False,False,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [14]:
# Use the final model to predict the `tow` for the submission_set_updated
submission_set_features = submission_set_updated.iloc[:,:-1]
submission_set['tow'] = final_model.predict(submission_set_features)

submission_set

[LightGBM] [Warning] lambda_l2 is set=0.46415888336127775, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.46415888336127775
[LightGBM] [Warning] lambda_l1 is set=0.166810053720005, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.166810053720005


,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow
0,248753821,2022-01-01,3b3de0f3ad0ee192513995c02f7bf7cf,LTFJ,Istanbul Sabiha Gokcen,TR,LFLL,Lyon,FR,2022-01-01T09:44:00Z,2022-01-01T12:48:33Z,B738,M,6351ec1b849adacc0cbb3b1313d8d39b,170,15,1122,69844.857740
1,248753822,2022-01-01,e06dd03d4a879ca37d9e18c1bd7cad16,EBBR,Brussels,BE,KJFK,New York JFK,US,2022-01-01T09:45:00Z,2022-01-01T17:49:51Z,A333,H,bdeeef3a675587d530de70a25d7118d2,470,15,3205,211826.293524
2,248754498,2022-01-01,2d3b1c962c78c4ebeef11bcd51b9e94c,KMIA,Miami,US,EGLL,London Heathrow,GB,2022-01-01T01:52:00Z,2022-01-01T09:55:16Z,B77W,H,5543e4dc327359ffaf5b9c0e6faaf0e1,473,10,3965,223474.382014
3,248757623,2022-01-01,81564432d3ee97c4bdf4cd8f006753dc,EGCN,Doncaster Sheffield,GB,LEAL,Alicante,ES,2022-01-01T08:20:00Z,2022-01-01T11:06:08Z,B38M,M,3922524069809ac4326134429751e26f,156,10,986,57240.176213
4,248763603,2022-01-01,84be079d7e660db105d91f600b4b3d59,EIDW,Dublin,IE,LFLL,Lyon,FR,2022-01-01T11:01:00Z,2022-01-01T13:00:43Z,A320,M,a73f82288988b79be490c6322f4c32ed,105,15,686,63915.538430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105954,258066302,2022-12-31,2d3b4446c4d05a25196a9d52cab936fb,LTFJ,Istanbul Sabiha Gokcen,TR,EKCH,Copenhagen,DK,2022-12-31T09:36:00Z,2022-12-31T13:12:17Z,B38M,M,6351ec1b849adacc0cbb3b1313d8d39b,201,15,1199,69157.723228
105955,258068609,2022-12-31,253fd692ed441fac523081471c067772,LOWW,Vienna,AT,KIAD,Washington Dulles,US,2022-12-31T09:49:00Z,2022-12-31T19:38:26Z,B763,H,5d407cb11cc29578cc3e292e743f5393,575,14,3937,176970.682815
105956,258068876,2022-12-31,c9fca302ca2e28acab0eb0bb1b46f11b,LTFM,iGA Istanbul,TR,LSZH,Zurich,CH,2022-12-31T09:25:00Z,2022-12-31T12:24:24Z,A321,M,6351ec1b849adacc0cbb3b1313d8d39b,154,25,988,73207.838510
105957,258064675,2022-12-31,00f96ad0e382476649574ba044c764fc,EHAM,Amsterdam,NL,EDDF,Frankfurt,DE,2022-12-31T10:04:21Z,2022-12-31T10:55:35Z,A320,M,f502877cab405652cf0dd70c2213e730,42,9,240,59151.688376


In [15]:
# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")
submission_set.to_csv(submission_file, index=False)